In [1]:
# Uncomment and install dependencies, if necessary
#!pip install html5lib
#!pip install webdriver_manager
#!pip install splinter
#!pip install lxml
#!pip install bs4

#Import dependencies
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np   
from bs4 import BeautifulSoup as bs  
import os 
import html5lib as html

# Extract
* ### Read CSV file for 1st data source: "Vaccination Coverage and Exemptions among Kindergartners"

In [2]:
coverage_exmpt_file = "Data\Vaccination_Coverage_and_Exemptions_among_Kindergartners.csv"
coverage_exmpt_file_df = pd.read_csv(coverage_exmpt_file, encoding="ISO-8859-1")
coverage_exmpt_file_df.head(100)

,Vaccine/Exemption,Dose,Geography Type,Geography,School Year,Estimate (%),Population Size,Percent Surveyed,Footnotes,Number of Exemptions,Survey Type
0,MMR,NaN,States,Alabama,2009-10,94.0,75358.0,100.0,â¥. *. **,NaN,Census
1,MMR,NaN,States,Alaska,2009-10,NaN,9641.0,76.4,NaN,NaN,Voluntary response
2,MMR,NaN,States,Arizona,2009-10,95.0,89287.0,93.9,NaN,NaN,Census
3,MMR,NaN,States,Arkansas,2009-10,98.3,34733.0,100.0,*,NaN,Census
4,MMR,NaN,States,California,2009-10,93.6,507191.0,100.0,*,NaN,Census
...,...,...,...,...,...,...,...,...,...,...,...
95,MMR,NaN,States,South Dakota,2010-11,NaN,NaN,NaN,NaN,NaN,NaN
96,MMR,NaN,States,Tennessee,2010-11,NaN,NaN,NaN,NaN,NaN,NaN
97,MMR,NaN,States,Texas,2010-11,NaN,NaN,NaN,NaN,NaN,NaN
98,MMR,NaN,National,U.S. Median,2010-11,NaN,NaN,NaN,NaN,NaN,NaN


* ### Filter for Exemptions and Remove Unnecessary Columns
* ### Drop NaN Values from 'Estimate (%)' and 'Survey Type' Columns
* ### Convert 'School Year' Column to Numeric and Filter for 2016-2019
* ### Save to 'Exemptions' Dataframe

In [3]:
exemptions_df = coverage_exmpt_file_df.loc[(coverage_exmpt_file_df['Vaccine/Exemption'] == 'Exemption') & (coverage_exmpt_file_df['Dose'] == 'Any Exemption'),['Geography','School Year','Estimate (%)','Survey Type']].copy()
exemptions_df.dropna(subset=['Estimate (%)'], inplace =True)
exemptions_df.rename(columns={"Estimate (%)": "Exempt_Estimate_Percent"}, inplace = True)
exemptions_df.dropna(subset=['Survey Type'], inplace =True)
exemptions_df["School Year"]=pd.to_numeric(exemptions_df["School Year"].str[:4])
exemptions_df=exemptions_df.loc[exemptions_df['School Year']>2015]
exemptions_df

,Geography,School Year,Exempt_Estimate_Percent,Survey Type
4762,Alabama,2016,0.7,Census
4763,Alaska,2016,6.8,Census
4764,Arizona,2016,5.1,Census
4765,Arkansas,2016,1.4,Voluntary response
4766,California,2016,1.1,Census
...,...,...,...,...
4964,Virginia,2019,1.7,Census
4965,Washington,2019,5.7,Census
4966,West Virginia,2019,0.1,Voluntary response
4967,Wisconsin,2019,5.7,Census


* ### Filter for Chicken Pox (Varicella) Coverage and Remove Unnecessary Columns
* ### Drop NaN Values from 'Estimate (%)' and Columns
* ### Convert 'School Year' Column to Numeric and Filter for 2016-2019
* ### Save to 'Vaccine' Dataframe

In [4]:
vaccine_df = coverage_exmpt_file_df.loc[coverage_exmpt_file_df['Vaccine/Exemption'] == 'Varicella, unknown disease history',['Dose','Geography','School Year','Estimate (%)','Survey Type']].copy()
vaccine_df.dropna(subset=['Estimate (%)'], inplace =True)
vaccine_df.rename(columns={"Estimate (%)": "Vaccinated_Estimate_Percent"}, inplace = True)
vaccine_df.dropna(subset=['Survey Type'], inplace =True)
vaccine_df.drop(columns=['Survey Type'], inplace=True)
vaccine_df["School Year"]=pd.to_numeric(vaccine_df["School Year"].str[:4])
vaccine_df=vaccine_df.loc[vaccine_df['School Year']>2015]
vaccine_df

,Dose,Geography,School Year,Vaccinated_Estimate_Percent
1524,1 Dose,Alabama,2016,93.8
1525,1 Dose,Arizona,2016,96.7
1526,1 Dose,California,2016,98.5
1527,1 Dose,Hawaii,2016,95.3
1528,1 Dose,Maine,2016,96.7
...,...,...,...,...
2158,2 Doses,Virginia,2019,93.3
2159,2 Doses,Washington,2019,92.7
2160,2 Doses,West Virginia,2019,97.8
2161,2 Doses,Wisconsin,2019,91.6


In [5]:
merged_df=exemptions_df.merge(vaccine_df,how='left', on=['Geography','School Year']).copy()
merged_df.rename(columns={'School Year':'Year'}, inplace=True)
merged_df.to_csv('Data/vaccine_exemptions.csv')
merged_df

,Geography,Year,Exempt_Estimate_Percent,Survey Type,Dose,Vaccinated_Estimate_Percent
0,Alabama,2016,0.7,Census,1 Dose,93.8
1,Alaska,2016,6.8,Census,2 Doses,88.9
2,Arizona,2016,5.1,Census,1 Dose,96.7
3,Arkansas,2016,1.4,Voluntary response,2 Doses,91.7
4,California,2016,1.1,Census,1 Dose,98.5
...,...,...,...,...,...,...
237,West Virginia,2019,0.1,Voluntary response,2 Doses,97.8
238,Wisconsin,2019,5.7,Census,1 Dose,NReq
239,Wisconsin,2019,5.7,Census,2 Doses,91.6
240,Wyoming,2019,3.5,Census,1 Dose,NReq


* ### Scrape data from HTML data
* ### Create paths to the html templates for each page

In [6]:
HTML_2016_template = os.path.join ("Data","2016-table2o.html")
HTML_2017_template = os.path.join ("Data","2017-table2o.html")
HTML_2018_template = os.path.join ("Data","2018-table2o.html")
HTML_2019_template = os.path.join ("Data","2019-table2r.html")

In [7]:
#Read HTML using pandas and export into tables
table_2016 = pd.read_html(HTML_2016_template)
table_2017 = pd.read_html(HTML_2017_template)
table_2018 = pd.read_html(HTML_2018_template)
table_2019 = pd.read_html(HTML_2019_template)

In [8]:
#Check length of each table list
print(len(table_2016),len(table_2017), len(table_2018), len(table_2019))

1 1 1 1


In [9]:
#Create dataframes for each table and add a year column
#2016
df2016 = table_2016[0]
df2016.columns = ["Reporting Area","Typhoid Fever","VNC-interm SA","VNC-resistant SA", "Varicella Morbidity","Varicella Mortality","Vibriosis Total","Vibriosis Confirmed","Vibriosis Probable"]
df2016['Year'] = "2016"
df2016.head()

,Reporting Area,Typhoid Fever,VNC-interm SA,VNC-resistant SA,Varicella Morbidity,Varicella Mortality,Vibriosis Total,Vibriosis Confirmed,Vibriosis Probable,Year
0,United States,376,108,—,8953,3,1273,1241,32,2016
1,New England,21,2,—,816,—,104,102,2,2016
2,Connecticut,7,—,—,113,—,17,17,—,2016
3,Maine,1,1,—,228,—,7,7,—,2016
4,Massachusetts,12,1,—,288,N,63,62,1,2016


In [10]:
#2017
df2017 = table_2017[0]
df2017.columns = ["Reporting Area","Typhoid Fever","VNC-interm SA","VNC-resistant SA", "Varicella Morbidity","Varicella Mortality","Vibriosis Total","Vibriosis Confirmed","Vibriosis Probable"]
df2017['Year'] = "2017"
df2017.head()

,Reporting Area,Typhoid Fever,VNC-interm SA,VNC-resistant SA,Varicella Morbidity,Varicella Mortality,Vibriosis Total,Vibriosis Confirmed,Vibriosis Probable,Year
0,United States,419,109,3,8775,2,2085,1497,588,2017
1,New England,20,2,1,912,1,153,140,13,2017
2,Connecticut,2,1,—,111,1,47,39,8,2017
3,Maine,—,—,—,198,—,7,7,—,2017
4,Massachusetts,15,1,1,386,—,72,71,1,2017


In [11]:
#2018
df2018 = table_2018[0]
df2018.columns = ["Reporting Area","Typhoid Fever","VNC-interm SA","VNC-resistant SA", "Varicella Morbidity","Varicella Mortality","Vibriosis Total","Vibriosis Confirmed","Vibriosis Probable"]
df2018['Year'] = "2018"
df2018.head()

,Reporting Area,Typhoid Fever,VNC-interm SA,VNC-resistant SA,Varicella Morbidity,Varicella Mortality,Vibriosis Total,Vibriosis Confirmed,Vibriosis Probable,Year
0,United States,401,85,—,8201,6,2964,1822,1142,2018
1,New England,23,1,—,933,—,185,147,38,2018
2,Connecticut,5,—,—,179,—,56,30,26,2018
3,Maine,—,—,—,252,—,14,14,—,2018
4,Massachusetts,17,1,—,289,—,83,79,4,2018


In [12]:
df2019 = table_2019[0]
df2019
df2019['Year'] = "2019"
df2019.head()

Reporting Area  \
                               Reporting Area   
0  U.S. Residents, excluding U.S. Territories   
1                                 New England   
2                                 Connecticut   
3                                       Maine   
4                               Massachusetts   

  Vancomycin-intermediate Staphylococcus aureus  \
  Vancomycin-intermediate Staphylococcus aureus   
0                                            76   
1                                             1   
2                                             —   
3                                             —   
4                                             1   

  Vancomycin-resistant Staphylococcus aureus Varicella morbidity  \
  Vancomycin-resistant Staphylococcus aureus Varicella morbidity   
0                                          3                8297   
1                                          1                 647   
2                                          —                 117   
3                                          —                  93   
4                                          1                 301   

  Varicella mortality Vibriosis                     Year  
  Varicella mortality     Total Confirmed Probable        
0                   6      2851      1651     1200  2019  
1                   —       182       142       40  2019  
2                   —        66        45       21  2019  
3                   —         9         8        1  2019  
4                   —        90        76       14  2019

In [13]:
#drop unnecessary columns
df2016.drop(columns=['Typhoid Fever','VNC-interm SA','VNC-resistant SA','Vibriosis Total','Vibriosis Confirmed','Vibriosis Probable'],inplace=True)
df2017.drop(columns=['Typhoid Fever','VNC-interm SA','VNC-resistant SA','Vibriosis Total','Vibriosis Confirmed','Vibriosis Probable'], inplace=True)
df2018.drop(columns=['Typhoid Fever','VNC-interm SA','VNC-resistant SA','Vibriosis Total','Vibriosis Confirmed','Vibriosis Probable'], inplace=True)
df2019.drop(columns=['Vancomycin-intermediate Staphylococcus aureus','Vancomycin-resistant Staphylococcus aureus','Vibriosis'], inplace= True)
df2019.columns=['Reporting Area', 'Varicella Morbidity', 'Varicella Mortality','Year']

In [14]:
#Merge dataframes into one dataframe
dataframes_list = [df2017, df2018, df2019]

#new dataframe to store append result
chickenpox_cases_df = df2016

# for df in dataframes_list:
chickenpox_cases_df=chickenpox_cases_df.append(df2017,ignore_index=True)
chickenpox_cases_df=chickenpox_cases_df.append(df2018,ignore_index=True)
chickenpox_cases_df=chickenpox_cases_df.append(df2019,ignore_index=True)
    # print(chickenpox_cases_df)
# chickenpox_cases_df.append(df2016)
chickenpox_cases_df.rename({'Reporting Area':'Geography'}, inplace=True)
chickenpox_cases_df.to_csv("Data\chix_cases.csv",index=False)